# A quick guide to implementing the model

The Jupyter notebooks provided in this project will walk users through the experimental procedures used for this particular paper. However, at it's core, the methodology can be broken down as follows.

We'll start by importing the following classes from the project's `./mod` folder.

In [ ]:
from shared.mutual_information.entropy import entropy
from shared.mod.LM.BERT.BERT import BERT
import torch

wv = BERT(special_tokens=False)
H = entropy(dim=-1)

Breaking these down, the class `wv` is an instance of a Hugging-Face, `"bert-base-uncased"` model, just packed and ready to be used. It also is set up to allow users to select which hidden layers they'd like to use to represent lexical meaning. The default is what's decribed in the paper--the 8th and 12th hidden layers. Note: you could swap out this model for any other word vector model that suits your specific research needs.

The class `H` implements the equations described in the paper in full. When you call it, it will go through all of those steps in a single go. I've included the verbiage of the section after this tutorial for reference.

I ran all of this code on a GPU processor off of an old laptop with an NVIDIA GEFORCE-900 graphics card. People who know GPUs know that the memory on this card is quite low, so the methods in the experiment are all deconstructed--I go through stages of

1. creating word vectors, before
2. sampling and comparing utterances using `H`, and finally
3. testing the outputs of stage (2).

Despite this, if you want to be up and running quickly, all you need is the following:

first, you need a corpus of data. We simulate that here with three strings.

In [ ]:
FOX_HEADLINE = "Vaccinated Americans a majority of COVID deaths for first time in August: analysis"
CNN_HEADLINE = "New subvariants, family gatherings may bring more Covid-19 after holiday, but experts don’t expect severe surge"
REUTERS_HEADLINE = "Latest on the worldwide spread of COVID-19"

From there, you can feed two sentences into the model to compare to one another like so:

In [ ]:
cnn_v_fox = H(
    wv(CNN_HEADLINE)[0],
    wv(FOX_HEADLINE)[0]
)

cnn_v_reuters = H(
    wv(CNN_HEADLINE)[0],
    wv(REUTERS_HEADLINE)[0]
)

note that we use index `[0]` here. That's because the BERT model that I pre-loaded for you returns both vectors and tokens for each vector. Vectors are index `[0]` in the output.

If you're pulling a _sample_ of multiple utterances, that would look more like the following when implemented:

In [ ]:
REUTERS_HEADLINE_SAMPLE = [REUTERS_HEADLINE, "COVID-19 Projected Surge Over Holiday Season, Experts Say", "COVID-19 Lab-Based Origin Debunked"]

REUTERS_HEADLINE_SAMPLE_VECS = torch.cat(
    [wv(headline)[0] for headline in REUTERS_HEADLINE_SAMPLE],
    dim=0
)

cnn_v_reuters_sample = H(
    wv(CNN_HEADLINE)[0],
    REUTERS_HEADLINE_SAMPLE_VECS
)

Note that in our analyses, we generate vectors for each indivdual utterance. We did this in order to maximize dissimilarities between utterances, as longer contexts may lead to vectors with higher similarity to one another. For our specific hypotheses, this would have been erroneous.

### Model Details

Imagine that an interlocutor is playing a kind of language reconstruction game. The interlocutor is given a single utterance from an individual, broken up into tokens. The interlocutor is then given a set of utterances also broken up into tokens from several utterances all taken from a number of members of some group. The interlocutor is then asked to take the groups' tokens and reconstruct an utterance that means something similar to the sentence they observed from the individual. This process can be repeated for the same original utterance using tokens from several different groups. In this scenario, reconstructed utterances that are more similar in meaning to the original utterance will have lower entropy. Reconstructed utterances that are either less similar or less intelligible will have higher entropy.

We operationalize this language game by calculating entropy for utterances using BERT word vectors \citep{devlin_bert_2019} to represent each token. This allows us to capture similarity between tokens that are semantically similar but are not a 1:1 mapping of the same word. Let $E_{xi}$ be the set of BERT word vectors for each token $w_i$ in a sentence $x$.
\begin{equation}
    E_{xi} = BERT(w_i \in x)
\end{equation}

the probability that two words are semantically similar to one another based on their word vectors is a function of their location in vector space \citep{pennington_glove_2014,mikolov_efficient_2013,devlin_bert_2019}. If a word vector were a point in space, words that are more semantically related to one another will be closer to one another. We use cosine similarity (CoS) to calculate the proximity between word vectors. Now, the probability of two word vectors meaning the same thing can be thought of in the following way: if word vectors put words that are more semantically similar to one another closer in space, the probability that a word/token $i$ from a sentence $x$ is semantically similar to a word/token $j$ from a sentence $y$ can be thought of colloquially as how likely you are to hit $xi$ if you were to throw a dart at $yj$. We quantify this intuition about probability and vector space in equation (SEE PAPER) using a Gaussian distribution with a location parameter $\mu=1.$ such that as the CoS value for the comparison of two word vectors approaches 1 we have maximum confidence that the two words mean the same thing, and a scale parameter $\sigma$.
\begin{equation}
    P(E_{xi} | E_{yj}) = P_{\mathcal{N}}\left( CoS(E_{xi},E_{yj}) \bigg|  \mu=1, \sigma \right)
\end{equation}
Think of $\sigma$ like the accuracy of the dart thrower, where lower $\sigma$ values equate to the dart thrower only hitting a word/token $xi$ if it is very close to $yj$ in word vector space.

However, we almost never have a reason to compare any one vector from a sentence $xi$ to any single vector from another sentence/distribution, $yj$. Instead, it’s better to ask how likely is a vector $xi$ conditioned on what we know about the total distribution $y$, in which there are $j$ tokens ($j \in y$). A priori, one way of posing this question is by asking “when we compare $xi$ to the entirety of the distribution $y$, which token $j \in y$ returns the maximum likelihood for $xi$ and what is the probability of $xi$ conditioned on that token?” We thus rewrite equation (1) as follows:

\begin{equation}
    P(E_{xi} | E_{y}) = P_{\mathcal{N}} \left( \max_{j} \left(CoS(E_{xi},E_{y}) \right) \bigg|  \mu=1, \sigma \right)
\end{equation}

\noindent This not only, renders $P(E_{xi} | E_{y})$ functionally equivalent to the term $P(w_i \in x | y)$ in equation (SEE PAPER), but also helps us control for variation in the length of utterances being analyzed. If one is calculating the entropy for an utterance $x$ that is 12 tokens long to an utterance or set of utterances $y_1$ that is 20 tokens long and yet another utterance or set of utterances $y_2$ that is 30 tokens long, it is possible that simply summing all comparisons--$P(E_{xi} | E_{yj})$ will result in higher entropy for $y_2$ than $y_1$ based solely on the difference in length between the two. By taking the maximum of the columns in both cases ($\max_j()$) we not only more directly ask how likely is the vector $E_{xi}$ based on the totality of either distribution $y_1$ or $y_2$, but we control for the effect of length on our entropy calculation.

From the perspective of a transformer language model like BERT, the only way that the function $\max_{j} \left(CoS(E_{xi},E_{y}) \right)$ can approach 1 is if there exists some overlapping, similar context between $x$ and $y$. Thus, in most cases there are three potential phenomena that increase the probability of $P(E_{xi} | E_y)$. Either (1) a number of lexical items in $j \in y$ tend to be semantically similar to $xi$ (because each lexical item $j \in y$ is embedded in a similar context to $xi$) such that any sample from the distribution $y$ will likely contain items that maximize $P(E_{xi} | E_y)$ (i.e. things that are semantically similar to $xi$ are common in the distribution $y$), (2) the distribution $y$ influenced the construction of $xi$ or vice-versa, or (3) the sample described by $y$ is large enough that something semantically approximating $xi$ eventually shows up in the data by sheer chance (which acts as a sort of \textit{de facto} null hypothesis). Because group members actively seek to increase similarity between each others’ idiolects in intragroup communication (1) and (2) are orders of magnitude more likely than (3) on most time scales.

Using this probability calculation we can generate an entropy for the entirety of an utterance $x$, by comparing the words vectors for words/tokens $i$ (i.e. all $i \in x$ or $xi$) and the distribution $y$.
\begin{equation}
    H( x ; y ) = -\sum_i P(E_{xi} | E_{y}) \log P(E_{xi} | E_{y})
\end{equation}